In [1]:
#import libraries
import pandas as pd
import numpy as np
import re

In [2]:
#open manufacturing workers data
df = pd.read_csv("data/manufacturing-workers-mexico-total.csv", skiprows=4)

In [3]:
#show data
df

,Periodos,"Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Total nacional /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Aguascalientes /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Baja California > Total /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Baja California > Ensenada /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Baja California > Mexicali /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Baja California > Tecate /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Baja California > Tijuana /f1 /a","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Coahuila de Zaragoza > Total /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Coahuila de Zaragoza > Acu�a /f1",...,"Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Tamaulipas > Total /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Tamaulipas > Matamoros /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Tamaulipas > Nuevo Laredo /f1","Manufacturas > Industria manufacturera, maquiladora y de serv

In [4]:
#drop final rows
df = df.drop([174,175,176])

In [5]:
#untidy data
df_melted = pd.melt(df, id_vars=["Periodos"], var_name ="State, City", value_name ="Manufacturing workers").copy()
df_melted

,Periodos,"State, City",Manufacturing workers
0,2007/07,"Manufacturas > Industria manufacturera, maquil...",1912745.0
1,2007/08,"Manufacturas > Industria manufacturera, maquil...",1929146.0
2,2007/09,"Manufacturas > Industria manufacturera, maquil...",1940574.0
3,2007/10,"Manufacturas > Industria manufacturera, maquil...",1953906.0
4,2007/11,"Manufacturas > Industria manufacturera, maquil...",1936734.0
...,...,...,...
10435,2021/08,"Manufacturas > Industria manufacturera, maquil...",139953.0
10436,2021/09,"Manufacturas > Industria manufacturera, maquil...",138520.0
10437,2021/10,"Manufacturas > Industria manufacturera, maquil...",140077.0
10438,2021/11,"Manufacturas > Industria manufacturera, maquil...",143237.0


In [6]:
rows = df_melted.shape[0]
rows

10440

In [7]:
df_melted["State, City"][220]

'Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Aguascalientes /f1'

In [8]:
#clean data (parentesis)
for i in range(rows):
    df_melted["State, City"][i]= re.sub(r'\([^)]*\)', '', df_melted["State, City"][i])

<ipython-input-8-c4ca17155f0d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_melted["State, City"][i]= re.sub(r'\([^)]*\)', '', df_melted["State, City"][i])


In [9]:
df_melted["State, City"][220]

'Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n  > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Aguascalientes /f1'

In [10]:
#clean data (other text)
df_clean= df_melted.replace("Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n  > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > ","",regex=True)
df_clean= df_clean.replace(" /f1","",regex=True)
df_clean= df_clean.replace(">",",",regex=True)
df_clean

,Periodos,"State, City",Manufacturing workers
0,2007/07,Total nacional,1912745.0
1,2007/08,Total nacional,1929146.0
2,2007/09,Total nacional,1940574.0
3,2007/10,Total nacional,1953906.0
4,2007/11,Total nacional,1936734.0
...,...,...,...
10435,2021/08,Otras entidades federativas,139953.0
10436,2021/09,Otras entidades federativas,138520.0
10437,2021/10,Otras entidades federativas,140077.0
10438,2021/11,Otras entidades federativas,143237.0


In [11]:
#split columns
df_clean["Month"] = df_clean["Periodos"].str.split('/', expand=True)[1]
df_clean["Year"] = df_clean["Periodos"].str.split('/', expand=True)[0]
df_clean["State"] = df_clean["State, City"].str.split(',', expand=True)[0]
df_clean["City"] = df_clean["State, City"].str.split(',', expand=True)[1]
df_clean.drop(["Periodos","State, City"], axis=1)

,Manufacturing workers,Month,Year,State,City
0,1912745.0,07,2007,Total nacional,None
1,1929146.0,08,2007,Total nacional,None
2,1940574.0,09,2007,Total nacional,None
3,1953906.0,10,2007,Total nacional,None
4,1936734.0,11,2007,Total nacional,None
...,...,...,...,...,...
10435,139953.0,08,2021,Otras entidades federativas,None
10436,138520.0,09,2021,Otras entidades federativas,None
10437,140077.0,10,2021,Otras entidades federativas,None
10438,143237.0,11,2021,Otras entidades federativas,None


In [12]:
#arrange columns
df_clean = df_clean[["Year","Month", "State","City", "Manufacturing workers"]]
df_clean

,Year,Month,State,City,Manufacturing workers
0,2007,07,Total nacional,None,1912745.0
1,2007,08,Total nacional,None,1929146.0
2,2007,09,Total nacional,None,1940574.0
3,2007,10,Total nacional,None,1953906.0
4,2007,11,Total nacional,None,1936734.0
...,...,...,...,...,...
10435,2021,08,Otras entidades federativas,None,139953.0
10436,2021,09,Otras entidades federativas,None,138520.0
10437,2021,10,Otras entidades federativas,None,140077.0
10438,2021,11,Otras entidades federativas,None,143237.0


In [13]:
df_clean["State"].unique()

array(['Total nacional', 'Aguascalientes', 'Baja California ',
       'Coahuila de Zaragoza ', 'Chihuahua ', 'Ciudad de M�xico',
       'Durango', 'Guanajuato ', 'Jalisco ', 'M�xico ', 'Nuevo Le�n ',
       'Puebla ', 'Quer�taro ', 'San Luis Potos� ', 'Sonora ',
       'Tamaulipas ', 'Veracruz de Ignacio de la Llave', 'Yucat�n ',
       'Otras entidades federativas'], dtype=object)

In [14]:
#clean other characters
df_clean = df_clean.replace("M�xico","Mexico",regex=True)
df_clean = df_clean.replace("Le�n","Leon",regex=True)
df_clean = df_clean.replace("Quer�taro","Queretaro",regex=True)
df_clean = df_clean.replace("Potos�","Potosi",regex=True)
df_clean = df_clean.replace("Yucat�n","Yucatan",regex=True)


In [15]:
df_clean["State"].unique()

array(['Total nacional', 'Aguascalientes', 'Baja California ',
       'Coahuila de Zaragoza ', 'Chihuahua ', 'Ciudad de Mexico',
       'Durango', 'Guanajuato ', 'Jalisco ', 'Mexico ', 'Nuevo Leon ',
       'Puebla ', 'Queretaro ', 'San Luis Potosi ', 'Sonora ',
       'Tamaulipas ', 'Veracruz de Ignacio de la Llave', 'Yucatan ',
       'Otras entidades federativas'], dtype=object)

In [16]:
df_clean.sample(20)

,Year,Month,State,City,Manufacturing workers
8364,2008,07,Sonora,Nogales,27618.0
3082,2017,11,Ciudad de Mexico,None,36198.0
5560,2021,05,Nuevo Leon,Apodaca,89742.0
10341,2013,10,Otras entidades federativas,None,124984.0
4834,2018,11,Mexico,Tlalnepantla de Baz,11850.0
8583,2012,04,Sonora,Otros municipios,50216.0
2847,2012,10,Chihuahua,Otros municipios,22955.0
4477,2018,02,Jalisco,Otros municipios,82277.0
9903,2020,10,Yucatan,Total,22059.0
5202,2020,07,Mexico,Otros municipios,93648.0


In [17]:
df_clean["City"].unique()

array([None, ' Total', ' Ensenada', ' Mexicali', ' Tecate', ' Tijuana /a',
       ' Acu�a', ' Ramos Arizpe', ' Saltillo', ' Torre�n',
       ' Otros municipios', ' Chihuahua', ' Ju�rez', ' Leon',
       ' Guadalajara', ' Zapopan', ' Tlalnepantla de Baz', ' Toluca',
       ' Apodaca', ' Guadalupe', ' Monterrey',
       ' San Nicol�s de los Garza', ' Santa Catarina', ' Puebla',
       ' Queretaro', ' San Luis Potosi', ' Hermosillo', ' Nogales',
       ' Matamoros', ' Nuevo Laredo', ' Reynosa', ' M�rida'], dtype=object)

In [18]:
#clean other characters
df_clean = df_clean.replace("Acu�a","Acuña",regex=True)
df_clean = df_clean.replace("Torre�n","Torreon",regex=True)
df_clean = df_clean.replace("Ju�rez","Juarez",regex=True)
df_clean = df_clean.replace("Nicol�s","Nicolas",regex=True)
df_clean = df_clean.replace("M�rida","Merida",regex=True)
df_clean = df_clean.replace(" /a","",regex=True)


In [19]:
df_clean["City"].unique()

array([None, ' Total', ' Ensenada', ' Mexicali', ' Tecate', ' Tijuana',
       ' Acuña', ' Ramos Arizpe', ' Saltillo', ' Torreon',
       ' Otros municipios', ' Chihuahua', ' Juarez', ' Leon',
       ' Guadalajara', ' Zapopan', ' Tlalnepantla de Baz', ' Toluca',
       ' Apodaca', ' Guadalupe', ' Monterrey',
       ' San Nicolas de los Garza', ' Santa Catarina', ' Puebla',
       ' Queretaro', ' San Luis Potosi', ' Hermosillo', ' Nogales',
       ' Matamoros', ' Nuevo Laredo', ' Reynosa', ' Merida'], dtype=object)

In [20]:
#remove none values
for z in range(rows):
    if df_clean["City"][z] == None:
        df_clean["City"][z] = "Total"

<ipython-input-20-cc734ebbda36>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["City"][z] = "Total"


In [21]:
#remove blank spaces at the end
for a in range(rows):
    df_clean["State"][a] = df_clean["State"][a].rstrip().lstrip()
    df_clean["City"][a] = df_clean["City"][a].rstrip().lstrip()

<ipython-input-21-185b7ceb8dc1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["State"][a] = df_clean["State"][a].rstrip().lstrip()
<ipython-input-21-185b7ceb8dc1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["City"][a] = df_clean["City"][a].rstrip().lstrip()


In [22]:
df_clean.sample(20)

,Year,Month,State,City,Manufacturing workers
4197,2009,04,Jalisco,Zapopan,28689.0
2724,2017,01,Chihuahua,Juarez,263909.0
2658,2011,07,Chihuahua,Juarez,178154.0
4356,2008,01,Jalisco,Otros municipios,58598.0
750,2012,01,Baja California,Mexicali,47552.0
10063,2019,08,Yucatan,Merida,16182.0
3456,2020,01,Guanajuato,Total,158521.0
7627,2019,08,San Luis Potosi,Total,87168.0
2171,2014,06,Coahuila de Zaragoza,Otros municipios,72147.0
8531,2007,12,Sonora,Otros municipios,51091.0


In [23]:
#flag border states and cities
border = ["Baja California", "Coahuila de Zaragoza", "Chihuahua", "Nuevo Leon", "Tamaulipas"]
f = len(border)
df_clean["Border"] = "NA"
for k in range(rows):
    p = 0
    for l in range(f):
        if df_clean["State"][k] == border[l]:
            p = p +1
    if p > 0:
        df_clean["Border"][k] = "Yes"
    else:
        df_clean["Border"][k] = "No"

<ipython-input-23-c499f2075a70>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["Border"][k] = "No"
<ipython-input-23-c499f2075a70>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["Border"][k] = "Yes"


In [24]:
df_clean.sample(20)

,Year,Month,State,City,Manufacturing workers,Border
4056,2012,01,Jalisco,Guadalajara,22089.0,No
1429,2010,08,Coahuila de Zaragoza,Acuña,24934.0,Yes
3244,2016,11,Durango,Total,44007.0,No
7058,2015,09,Queretaro,Total,81108.0,No
1130,2014,09,Baja California,Tijuana,188078.0,Yes
10076,2020,09,Yucatan,Merida,7299.0,No
7641,2020,10,San Luis Potosi,Total,87009.0,No
1969,2012,02,Coahuila de Zaragoza,Torreon,26084.0,Yes
1927,2008,08,Coahuila de Zaragoza,Torreon,28240.0,Yes
6926,2019,03,Puebla,Otros municipios,57913.0,No


In [25]:
df_clean.groupby("Border")["Border"].agg("count")

Border
No     5742
Yes    4698
Name: Border, dtype: int64

In [26]:
df_clean.query("State == 'Baja California'")

,Year,Month,State,City,Manufacturing workers,Border
348,2007,07,Baja California,Total,257798.0,Yes
349,2007,08,Baja California,Total,260827.0,Yes
350,2007,09,Baja California,Total,261260.0,Yes
351,2007,10,Baja California,Total,262512.0,Yes
352,2007,11,Baja California,Total,259918.0,Yes
...,...,...,...,...,...,...
1213,2021,08,Baja California,Tijuana,266351.0,Yes
1214,2021,09,Baja California,Tijuana,268704.0,Yes
1215,2021,10,Baja California,Tijuana,266005.0,Yes
1216,2021,11,Baja California,Tijuana,262993.0,Yes


In [27]:
#Export to CSV
df_clean.to_csv("data/manufacturing-workers-mexico-refined.csv", index = False, encoding = "UTF8")